#5 MEMORY

#5.0 ConversationBufferMemory -  save all return messages

In [4]:
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory()
memory.save_context({"input":"Hi!"},{"output":"How are you?"})
memory.load_memory_variables({})


{'history': 'Human: Hi!\nAI: How are you?'}

In [ ]:
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory(
    return_messages=True
)

memory.save_context({"input":"Hi!"},{"output":"How are you?"})
memory.load_memory_variables({})

memory.save_context({"input":"Hi!"},{"output":"How are you?"})
memory.load_memory_variables({})

#5.1 ConversationBufferWindowMemory - limit the memory buffer

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=2,
)


def add_message(input, output):
    memory.save_context({"input":input},{"output":output})

    
add_message(1,1)
add_message(2,2)
memory.load_memory_variables({})

In [ ]:
add_message(3,3)
memory.load_memory_variables({})

#5.2 ConversationSummaryMemory - summarize the conversation and cost as memory to run

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [14]:
add_message("South Kddorea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()

#5.3 ConversationSummaryBufferMemory - Mix with Conversation and Buffer -  Keep track of recent messages and after reach certain maximum limit, rest of messagee will be summarized.

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=100,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
get_history()

In [37]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [ ]:
get_history()

In [40]:
add_message("How far is Korea from Argentina?", "I don't know! Super far!")

In [ ]:
get_history()

In [42]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [ ]:
get_history()

In [44]:
add_message("How far is Brazil from Argentina?", "I don't know! Super far!")

In [ ]:
get_history()

#5.4 ConversationKGMemory(Conversation Knowledge Graph Memory) - Extract the entity of conversation

In [46]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationKGMemory

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message("Hi I'm Nicolas, I live in South Korea", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"input": "who is Nicolas"})

In [48]:
add_message("Nicolas likes kimchi", "Wow that is so cool!")

In [ ]:
memory.load_memory_variables({"inputs": "what does nicolas like"})

#5.5 Memory on LLMChain -  How to Plug memory into chain ***

Off-the-shelf chain -> memory return string

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=80,
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")

In [ ]:
chain.predict(question="What is my name?")

위 코드의 문제점은 전에 사용한 메세지 기록이 현재 프롬프트에 포함이 안되어서 질문에 답을 못하는 상황이지먄

메모리에는 메세지가 기록되어 있는 상태이므로 

코딩상 해야 될 일은 프롬프트에 Message history를 넣어주면 문제 해결됨

In [ ]:
memory.load_memory_variables({})

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
)

template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

In [ ]:
chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")

In [ ]:
chain.predict(question="What is my name?")

In [ ]:
chain.predict(question="where am I living?")

In [ ]:
memory.load_memory_variables({})

#5.6 Chat Based Memory LLMChain

Off-the-shelf chain -> memory return message

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

In [ ]:
chain.predict(question="My name is Nico")

In [ ]:
chain.predict(question="I live in Seoul")

In [ ]:
chain.predict(question="What is my name?")

#5.7 LCEL Based Memory -> Custom Chain

Without RunnablePassthrough - invoke를 call 할때마다 메모리를 불러와야 함

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

chain = prompt | llm

chain.invoke({
    "chat_history": memory.load_memory_variables({})["history"],
    "question": "My name is nico"
    })

Better way Using RunnablePassthrough

In [9]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])


def load_memory(_):
#def load_memory(input):
    #print(input)
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm    line 1

chain.invoke({"chat_history": load_memory((),"question": question})       line 2


이 코드는 위의 코드와 같은 코드이지만 

위의 코드는 프롬프트가 format되기 전에 함수를 실행하는것을 허락해 준다.

line 1의 load_memory 함수의 결과값을 line 2 의 chat_history input으로 입력하라고 알려줄수 있으며

그 결과값을 invoke 함수의 question variable 과 같이 prompt로 전달해서 

MessagesPlaceholder(variable_name="history")에 넣어주게 되므로

prompt는 chat_history와 question varialbles 둘 다 받을 수 있게 된다.

In [ ]:
invoke_chain("My name is nico")

In [ ]:
invoke_chain("What is my name?")